## 3 rings Samwise target 10G

In [1]:
import numpy as np
import magpylib as magpy
from maggeometry import n_rings, four_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid

In [2]:
Br = 1.09e3
mag_dir = (0,0,1)
mirror1_z = False
ringset_1_config = [Br, mag_dir, mirror1_z]

r = 1.09e3
mag_dir = (0,0,1)
mirror2_z = True
ringset_2_config = [Br, mag_dir, mirror2_z]
# innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2
bounds = ((1, 150), (2, 20), (2, 20),
          (1, 150), (2, 20), (2, 20), (1, 300))

In [3]:
def get_grid_mag_and_target_nonuniformity(magnets, grid, grid_res, target=10, use_z=False):
    mT_to_G = 10
    B = magnets.getB(grid)
    G = B * mT_to_G
    # find magnitude of the b-field at all points over grid
    Gmag = 0
    if use_z:
        Gmag = G[:,:,2]
    else:
        Gmag = np.linalg.norm(G, axis=2)
#     Gmag = np.sqrt(G[:,:,0]**2 + G[:,:,1]**2 + G[:,:,2]**2)
#     Gmag = G[:,:,2]
    # find magnitude of b-field at center of grid
    mid_id = int(grid_res/2)
    Gcenter = Gmag[mid_id][mid_id]
    # calculate nonuniformity with respect to center b-field
    Gnon = np.abs((Gmag - Gcenter)/Gcenter)
    Gtarget_non = np.abs((Gmag - target)/target)
#     Gtarget_non = np.abs((Gmag - target)/target)
#     max_nonuniformity = np.max(Gnon)
    av_nonuniformity = np.sum(Gnon) / grid_res**2
    av_target_nonuniformity = np.sum(Gtarget_non) / grid_res**2
#     return Gmag, Gnon, Gcenter, av_nonuniformity, av_target_nonuniformity, max_nonuniformity
    return Gcenter, av_nonuniformity, av_target_nonuniformity

In [4]:
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
threshold = 0.1 # in G
target = 10
def obj1_nonuniformity(x):
    obj_ring_config=[[*ringset_1_config, *x[0:3], 0], [*ringset_2_config, *x[3:7]]]
    magnets = n_rings(obj_ring_config)
    center_field, av_nonuniformity, av_target_nonuniformity = get_grid_mag_and_target_nonuniformity(magnets, grid, grid_res, target)
    if (abs(center_field - target) > threshold):
        return av_target_nonuniformity
    return av_nonuniformity

In [ ]:
import numpy as np
from scipy.optimize import minimize, Bounds
from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid
from alive_progress import alive_bar

options = {'disp': True, 'fatol': 1e-18, 'maxiter': 5e3, 'adaptive': True}
method = "Nelder-Mead"

shots = 50
results_x = []
results_fun = []
guesses_x0 = []
with alive_bar(shots, force_tty=True) as bar:
    for i in range(shots):
        x0 = np.zeros(len(bounds))
        for b in range(len(bounds)):
            x0[b] = np.random.uniform(low=bounds[b][0], high=bounds[b][1])
        res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)
        guesses_x0.append(x0)
        results_x.append(res.x)
        results_fun.append(res.fun)
        bar()

print("Objective function values:")
print(results_fun)
print("Result parameters:")
print(results_x)
print("Starting guesses:")
print(guesses_x0)

|                                        | ▆█▆ 0/50 [0%] in 1:26 (~0s, 0.0/s) 

In [11]:
from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results_g_center = []
results_nonun = []
for x in results_x:
    obj_ring_config=[[*ringset_1_config, *x[0:4]]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    results_g_center.append(center_field)
    results_nonun.append(av_nonuniformity)
print(results_g_center)
print(results_nonun)

[9.90000000000208, 9.900000000006681, 10.099994848226928, 9.900000000120706, 9.899999945040108, 9.899999993247182, 9.900000000002054, 9.900000000076012, 10.100000000000279, 9.900000000000515, 9.900000000000514, 9.900000000125324, 10.100000000000275, 9.948321738674633, 10.100000000000264, 9.900000000000515, 10.100000000001046, 9.900000000005136, 9.900000208560407, 9.899999978652687, 10.100000000000279, 10.099997731850259, 9.900000021272826, 10.100000000001819, 10.099981131752891, 10.100000000000275, 10.100000000001046, 10.100000000000275, 9.900000010955615, 9.900000000001263, 10.099999969423052, 9.899999999998954, 10.100000000000279, 10.099998490692208, 9.9000000000036, 10.100000000000335, 10.099998851082734, 9.900000000351827, 9.900000000002063, 9.900000000000517, 9.900000000000514, 10.100000000000279, 9.900000040130728, 9.900000000002063, 10.099999290354233, 10.09731252742208, 9.900000000000517, 10.100000000000193, 10.100000000000279, 9.900000666565338]
[1.4562362742037886e-05, 1.3035

In [12]:
from maghelper import make_opt_res_csv
import datetime
today = datetime.datetime.today().strftime('%Y_%m_%d')
make_opt_res_csv('results/3 rings/{today}_samwise_3rings_1090T_', results_fun, results_g_center, results_x, guesses_x0)

Number of unique results: 50
    nonuniformity  center_field_gauss  r_1_innerrad  r_1_width  r_1_thickness  \
0    8.362246e-07           10.097313    149.999999  16.200758       8.876742   
1    9.291245e-07            9.948322    149.999990   7.956250      16.917564   
2    9.293194e-07           10.099981    149.999949   7.936917      17.216277   
3    9.399705e-07           10.099998    149.999992   7.038522      19.310594   
4    9.429695e-07           10.099995    150.000000   6.785950      20.000000   
5    1.349277e-06            9.900000    132.553890  15.571375       7.132045   
6    1.395957e-06           10.099998    133.364048  11.473119       9.706522   
7    1.515003e-06            9.900000    132.389768   7.773048      13.489552   
8    1.649939e-06            9.900000    123.594702  19.588936       5.119061   
9    1.673558e-06            9.900000    130.015817   5.785337      17.257624   
10   1.819443e-06            9.900000    127.745650   4.791058      19.999996   

(array([[8.36224554e-07, 1.00973125e+01, 1.49999999e+02, 1.62007584e+01,
         8.87674249e+00, 2.67512250e+02, 1.36967952e+02, 1.14887574e+01,
         7.24096584e+00, 2.91306804e+02],
        [9.29124490e-07, 9.94832174e+00, 1.49999990e+02, 7.95625035e+00,
         1.69175637e+01, 2.60959736e+02, 1.49483796e+02, 1.03453687e+01,
         1.64996005e+01, 2.24956909e+02],
        [9.29319441e-07, 1.00999811e+01, 1.49999949e+02, 7.93691720e+00,
         1.72162766e+01, 2.60956953e+02, 1.13940053e+02, 5.64729983e+00,
         1.23061204e+01, 2.10484896e+02],
        [9.39970547e-07, 1.00999977e+01, 1.49999992e+02, 7.03852226e+00,
         1.93105945e+01, 2.60307630e+02, 9.35156653e+01, 7.96492632e+00,
         1.97710474e+01, 2.56809968e+02],
        [9.42969483e-07, 1.00999948e+01, 1.50000000e+02, 6.78594981e+00,
         2.00000000e+01, 2.60132130e+02, 8.20951646e+01, 2.37687325e+00,
         1.67769979e+01, 2.98517895e+02],
        [1.34927738e-06, 9.90000000e+00, 1.32553890e+02, 1.5